In [1]:
#load libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import numpy as np
import time
import random
import pickle
import re
import requests
import csv
import json


In [2]:
# Initialize Chrome Options
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})  # Disable images


In [3]:
# Configure Chrome anti-bot measures
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--disable-blink-features=AutomationControlled")


In [4]:
# Initialize WebDriver with Chrome options
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [5]:
# Define paths for cookies and initial URLs
cookie_file_path = "cookies.pkl"
base_url = "https://finance.yahoo.com/"
sectors_url = "https://finance.yahoo.com/sectors/basic-materials/"

In [6]:
# Save cookies function
def save_cookies(driver, path):
    with open(path, "wb") as file:
        pickle.dump(driver.get_cookies(), file)

In [7]:
# Load cookies function with domain check
def load_cookies(driver, path):
    try:
        with open(path, "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                if "domain" in cookie and cookie["domain"] in driver.current_url:
                    driver.add_cookie(cookie)
            return True
    except FileNotFoundError:
        return False

In [8]:
# Initialize an empty list to hold industry names
industry_names = []

def gather_industry_names(driver):
    global industry_names
    # Reload sectors page if not on it
    if driver.current_url != sectors_url:
        print("Returning to sectors page...")
        driver.get(sectors_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr.yf-k3njn8"))
        )
        time.sleep(1)  # Short delay for additional page loading

    # Select industry rows after page load
    industry_rows = driver.find_elements(By.CSS_SELECTOR, "tr.yf-k3njn8")
    print(f"Found {len(industry_rows)} industries.")

    # Extract and clean names of the first 10 industries
    industry_names = [
        row.find_element(By.CSS_SELECTOR, "td.name").text for row in industry_rows[2:len(industry_rows)]#2:
    ]
    print("Collected industry names:", industry_names)

In [9]:
def generate_urls():
    # Generate URLs using industry names, replacing spaces with dashes and removing '&'
    updated_urls = [
        sectors_url + name.lower().replace('&', '').replace(' ', '-').replace('--', '-') + '/' 
        for name in industry_names
    ]
    return updated_urls


In [10]:
# Function to extract "/quote/.../" hrefs from each URL in the list
def extract_quote_links(driver, urls):
    quote_links = []  # List to hold fully qualified extracted links

    for index, url in enumerate(urls):
        print(f"Accessing URL {index + 1}/{len(urls)}: {url}")
        driver.get(url)
        time.sleep(5)  # Delay to allow page load; adjust as needed for speed optimization

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all hrefs that match "/quote/.../"
        matched_links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            if re.match(r"^/quote/.+/$", href):
                full_url = urljoin(base_url, href)  # Prepend base URL
                matched_links.append(full_url)

            # Stop after collecting the first 10 unique links
            if len(matched_links) >= 10:
                break

        quote_links.extend(matched_links)

        print(f"Extracted {len(matched_links)} links from {url}")

    return quote_links

In [11]:
# Save extracted links to CSV
def save_links_to_csv(links, filename="extracted_links.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Link"])  # Write header
        for link in links:
            writer.writerow([link])
    print(f"Links saved to {filename}")

In [12]:
# Retry function for operations
def retry_operation(func, retries=3, delay=5, *args, **kwargs):
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(delay)
    print("Operation failed after retries.")
    return None

In [13]:
# Fetch stock data for a single ticker
def fetch_stock_data(ticker, period1, period2, interval="1d", cookies=None, headers=None):
    base_url = "https://query1.finance.yahoo.com/v8/finance/chart/"
    query_url = (
        f"{base_url}{ticker}?period1={period1}&period2={period2}"
        f"&interval={interval}&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US"
    )
    
    try:
        response = requests.get(query_url, cookies=cookies, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Extract timestamps and close prices
        timestamps = data['chart']['result'][0]['timestamp']
        close_prices = data['chart']['result'][0]['indicators']['quote'][0]['close']
        
        # Convert timestamps to datetime
        timestamps = pd.to_datetime(timestamps, unit='s')

        # Filter for the last 100 days
        timestamps = timestamps[-100:]
        close_prices = close_prices[-100:]
        
        return timestamps, close_prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None, None

In [14]:
# Function to fetch data for all tickers
def fetch_all_tickers_data(links, period1, period2, interval="1d", cookies=None, headers=None):
    # Generate a fixed index for the last 100 days with time set to 15:45
    target_timestamps = pd.date_range(
        end=pd.to_datetime(period2, unit="s"),
        periods=100,
        freq="D"
    ).normalize() + pd.Timedelta(hours=15, minutes=49)  # Set time to 15:49
    
    # Temporary dictionary to store data
    ticker_data = {"timestamp": target_timestamps}

    for link in links:
        try:
            # Extract the ticker symbol
            ticker = link.split('/')[-2]
            print(f"Fetching data for {ticker}...")
            
            # Fetch data for the ticker
            timestamps, close_prices = fetch_stock_data(
                ticker, period1, period2, interval, cookies, headers
            )
            
            if timestamps is not None and close_prices is not None:
                # Create a Series for the close prices with timestamps as the index
                series = pd.Series(data=close_prices, index=timestamps)
                
                # Reindex to align with the target timestamps
                reindexed_series = series.reindex(target_timestamps, fill_value=np.nan)
                
                # Store the reindexed data
                ticker_data[ticker] = reindexed_series.values
        except Exception as e:
            print(f"Error processing {link}: {e}")

    # Convert the dictionary to a DataFrame
    aggregated_data = pd.DataFrame(ticker_data)
    return aggregated_data

In [15]:
# Save the aggregated data to CSV
def save_data_to_csv(data, filename="stock_data.csv"):
    data.to_csv(filename, index=False)
    print(f"Stock data saved to {filename}")

In [16]:
def fetch_all_tickers_data(links, period1, period2, interval="1d", cookies=None, headers=None):
    # Generate a fixed index for the last 100 days at 13:30
    target_timestamps = pd.date_range(
        end=datetime.now().replace(hour=13, minute=30, second=0),
        periods=100,
        freq="D"
    )
    
    # Temporary dictionary to store data
    ticker_data = {"timestamp": target_timestamps}


    for link in links:
        try:
            # Extract the ticker symbol
            ticker = link.split('/')[-2]
            print(f"Fetching data for {ticker}...")
            
            # Fetch data for the ticker
            timestamps, close_prices = fetch_stock_data(
                ticker, period1, period2, interval, cookies, headers
            )
            
            if timestamps is not None and close_prices is not None:
                print(f"Fetched {len(close_prices)} close prices for {ticker}.")
                # Create a Series for the close prices with timestamps as the index
                series = pd.Series(data=close_prices, index=timestamps)
                
                # Reindex to align with the target timestamps
                reindexed_series = series.reindex(target_timestamps, fill_value=np.nan)
                
                # Store the reindexed data
                ticker_data[ticker] = reindexed_series.values
            else:
                print(f"Warning: No data available for {ticker}.")
                ticker_data[ticker] = [np.nan] * len(target_timestamps)
        except Exception as e:
            print(f"Error processing {link}: {e}")
            ticker_data[ticker] = [np.nan] * len(target_timestamps)

    # Convert the dictionary to a DataFrame
    aggregated_data = pd.DataFrame(ticker_data)
    return aggregated_data

In [17]:
# Main function to handle navigation
def main():
    print("Navigating to Yahoo Finance homepage...")
    driver.get(base_url)
    time.sleep(2)
    
    # Handle cookies if they exist
    if load_cookies(driver, cookie_file_path):
        print("Cookies loaded successfully.")
        driver.refresh()
    else:
        print("No cookies found. Accepting cookies manually.")
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept all')]"))
            )
            accept_button.click()
            time.sleep(2)
            save_cookies(driver, cookie_file_path)
            print("Cookies accepted and saved.")
        except Exception as e:
            print("Error handling cookies:", e)
    
    # Navigate to Basic Materials sector
    print("Navigating to Basic Materials sector...")
    driver.get(sectors_url)
    time.sleep(2)
    
    # Navigate to each industry and collect company links
    #navigate_to_industry()
    gather_industry_names(driver)
    updated_urls = generate_urls()

    print("List of updated URLs for the first 10 industries:")
    for url in updated_urls:
        print(url)

    # Extract quote links from each updated URL
    print("Extracting /quote/.../ links from each industry page...")
    extracted_links = retry_operation(extract_quote_links, retries=3, driver=driver, urls=updated_urls)
    if not extracted_links:
        print("Failed to extract links. Exiting.")
        return
        
    print("Collected full quote links:") if extracted_links else print("Links could not be collected")

    # Save the extracted links
    save_links_to_csv(extracted_links)
    print("Extraction and saving complete.")
    
    for link in extracted_links:
        print(link)

    # Fetch stock data using Yahoo AJAX endpoint
    print("Fetching stock data for all links...")
    
    # Define a fixed timestamp range
    period1 = 1724080800  # Example start timestamp
    period2 = 1733104800  # Example end timestamp

    # Define headers and cookies
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    }

    cookies = {
        "GUC": "AQABCAFnJy1nV0IgvASK&s=AQAAAJB-qKHi&g=ZyXotw",
        "A1": "d=AQABBK3oJWcCEL768yJfkpdKJoPN52K9l3QFEgABCAEtJ2dXZ7u9b2UBAiAAAAcIpuglZ8iQvhs&S=AQAAAq11kqwbV-Cl4YdEYNtEpg8",
        "A3": "d=AQABBK3oJWcCEL768yJfkpdKJoPN52K9l3QFEgABCAEtJ2dXZ7u9b2UBAiAAAAcIpuglZ8iQvhs&S=AQAAAq11kqwbV-Cl4YdEYNtEpg8",
        "A1S": "d=AQABBK3oJWcCEL768yJfkpdKJoPN52K9l3QFEgABCAEtJ2dXZ7u9b2UBAiAAAAcIpuglZ8iQvhs&S=AQAAAq11kqwbV-Cl4YdEYNtEpg8",
        "cmp": "t=1733097066&j=1&u=1---&v=54",
        "EuConsent": "CQHdjMAQHdjMAAOACKENBRFgAAAAAAAAACiQAAAAAAAA",
        "PRF": "t=LIN%252BGC%253DF",
    }

    # Fetch and save stock data
    stock_data = fetch_all_tickers_data(extracted_links, period1, period2, interval="1d", cookies=cookies, headers=headers)
    save_data_to_csv(stock_data)
    print("Data fetching and saving complete.")

    


In [18]:
# Run the main function
main()

Navigating to Yahoo Finance homepage...
Cookies loaded successfully.
Navigating to Basic Materials sector...
Found 16 industries.
Collected industry names: ['Specialty Chemicals', 'Gold', 'Building Materials', 'Copper', 'Steel', 'Agricultural Inputs', 'Chemicals', 'Other Industrial Metals & Mining', 'Lumber & Wood Production', 'Aluminum', 'Other Precious Metals & Mining', 'Coking Coal', 'Paper & Paper Products', 'Silver']
List of updated URLs for the first 10 industries:
https://finance.yahoo.com/sectors/basic-materials/specialty-chemicals/
https://finance.yahoo.com/sectors/basic-materials/gold/
https://finance.yahoo.com/sectors/basic-materials/building-materials/
https://finance.yahoo.com/sectors/basic-materials/copper/
https://finance.yahoo.com/sectors/basic-materials/steel/
https://finance.yahoo.com/sectors/basic-materials/agricultural-inputs/
https://finance.yahoo.com/sectors/basic-materials/chemicals/
https://finance.yahoo.com/sectors/basic-materials/other-industrial-metals-mining

In [19]:
# Close the driver
print("Script complete. Closing the browser.")
driver.quit()

Script complete. Closing the browser.
